In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
test_df=pd.read_csv('./output/train_hypothesis_evidences.csv')
test_df

,hypothesis,evidences,label
0,All the primary trial participants do not rece...,"[""INTERVENTION 1: "", "" Diagnostic (FLT PET)"",...",Contradiction
1,"Patients with Platelet count over 100,000/mm¬¨...","["" PATIENT CHARACTERISTICS:"", "" ANC 1,500/m...",Contradiction
2,Heart-related adverse events were recorded in ...,"[""Adverse Events 1:"", "" Supraventricular tach...",Entailment
3,Adult Patients with histologic confirmation of...,"[""Inclusion Criteria:"", "" Patients with histo...",Contradiction
4,Laser Therapy is in each cohort of the primary...,"[""INTERVENTION 1: "", "" Laser Therapy Alone"", ...",Contradiction
...,...,...,...
1695,"Adequate blood, kidney, and hepatic function a...","[""Inclusion Criteria:"", "" Postmenopausal wome...",Entailment
1696,The Ridaforolimus + Dalotuzumab + Exemestane g...,"[""Outcome Measurement: "", "" 1. Progression-fr...",Contradiction
1697,The only difference between the interventions ...,"[""INTERVENTION 1: "", "" Prone"", ""Prone positio...",Entailment
1698,Patients must have a white blood cell count ab...,"["" WBC > 1,500/mm\u00b3""]",Entailment


In [3]:
hypothesis_lst=test_df['hypothesis'].values.tolist()
len(hypothesis_lst)

1700

In [4]:
evidence_lst=test_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

1700

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=test_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

1700

In [6]:
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import T5Tokenizer, T5ForConditionalGeneration

text_tok = T5Tokenizer.from_pretrained("t5-base")
text_clf = T5ForConditionalGeneration.from_pretrained("t5-base")

/home/x20779zz/.local/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [7]:
class InputSequence:
    
    def __init__(self,tok,l_text,l_text2,l_label,batch_size=64,gpu=True,task_prefix = "Detect entailment: "):
        
        self.data_len=len(l_text)
        self.data_idx=[i for i in range(self.data_len)]
        self.texts=tok([task_prefix+' '.join([a,b]) for a,b in zip(l_text,l_text2)],
                       padding="longest",
                       max_length=512,#max_source_length,
                       truncation=True,
                       return_tensors="pt",
                      )
        self.labels=tok(["Entailment" if lab==1 else "Contradiction" for lab in l_label],
                         padding="longest",
                         max_length=128,#,
                         truncation=True,
                         return_tensors="pt",
                        )
        print('tokenize done')
        
        self.batch_size=batch_size
        self.gpu=gpu
        
    def on_epoch_end(self):
        random.shuffle(self.data_idx)
        
    def __getitem__(self,i):
        start=i*self.batch_size
        batch_idx=self.data_idx[start:min(start+self.batch_size,self.data_len)]
        
        return_texts=dict([(k,self.texts[k][batch_idx]) for k in self.texts])
        return_labels=dict([(k,self.labels[k][batch_idx]) for k in self.labels])
        
        if self.gpu:
            return_texts=dict([(k,return_texts[k].cuda()) for k in return_texts])
            return_labels=dict([(k,return_labels[k].cuda()) for k in return_labels])
        
        return return_texts,return_labels
    
    def __len__(self):
        return math.ceil(1.0*self.data_len/self.batch_size)
    

In [8]:
testing_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,batch_size=16,gpu=True)

tokenize done


In [9]:
scores=[]
#T5
model_names=['t5-base']+[
    './output/clf_models/t5-base_epoch_{}.pt'.format(format(epoch,'05d'))
    for epoch in range(10)
]
for model_name in model_names:
    scores.append([])
    clf=T5ForConditionalGeneration.from_pretrained(model_name).cuda()
    with torch.no_grad():
        for batch in range(len(testing_data)):
            batch_texts,batch_labels=testing_data[batch]
            output_texts=text_tok.batch_decode(
                clf.generate(
                    input_ids=batch_texts["input_ids"],
                    attention_mask=batch_texts["attention_mask"],
                    do_sample=False,  # disable sampling to test if batching affects output
                ),
                skip_special_tokens=True
            )
            # print(output_texts)
            scores[-1].append([[0.0,1.0] if t=='Entailment' else [1.0,0.0] for t in output_texts])
            print('model:',model_name,'batch:',batch,end='\r')
    scores[-1]=np.concatenate(scores[-1],axis=0)
    clf.cpu()

/home/x20779zz/.local/lib/python3.9/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2022-11-29 17:51:32.861722: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


/home/x20779zz/.local/lib/python3.9/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [10]:
from sklearn.metrics import average_precision_score,f1_score,precision_score,recall_score,accuracy_score

y_true=label_lst
results=[]
for epoch in range(len(scores)):
    y_prob=scores[epoch][:,1]
    y_pred=[1 if a>0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch==0 else epoch,
        average_precision_score(y_true,y_prob),
        f1_score(y_true,y_pred),
        precision_score(y_true,y_pred),
        recall_score(y_true,y_pred),
        accuracy_score(y_true,y_pred)
    ])

import pandas as pd

pd.DataFrame(results,columns=['epoch','AVG_PREC','F1','PREC','REC','ACC'])

/home/x20779zz/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,epoch,AVG_PREC,F1,PREC,REC,ACC
0,pretrained,0.500000,0.000000,0.000000,0.000000,0.500000
1,1,0.501774,0.585868,0.502525,0.702353,0.503529
2,2,0.500590,0.491607,0.501222,0.482353,0.501176
3,3,0.505091,0.532810,0.509110,0.558824,0.510000
4,4,0.503953,0.321168,0.516971,0.232941,0.507647
5,5,0.521498,0.464481,0.553746,0.400000,0.538824
6,6,0.525771,0.380192,0.592040,0.280000,0.543529
7,7,0.537503,0.593819,0.559252,0.632941,0.567059
8,8,0.548400,0.569175,0.587719,0.551765,0.582353
9,9,0.564558,0.606702,0.606345,0.607059,0.606471
